In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## LangChain Agent 생성

In [3]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults


tools = [TavilySearchResults(max_results = 1)]

# 프롬프트 로드
prompt = hub.pull("hwchase17/openai-functions-agent")

# LLM 모델 정의
llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)

# Agent 생성
agent = create_openai_functions_agent(llm, tools, prompt)

## GraphState 정의

1. **input**: 사용자의 입력
2. **chat_hisotry**: 지난 대화 기록 메시지
3. **intermediate_steps**: 매 스텝 에이전트의 행동과 관측 결과를 저장하는 리스트
4. **agent_outcome**: 에이전트의 결과. AgentAction, AgentFinish 중 하나.

In [5]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    input: str
    chat_history: List[BaseMessage]
    # agnet_outcome 변수가 초기에는 None 값을 가질 수 있어야 하므로, 타입 정의에 None을 유효한 타입으로 포함
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # 새로운 값이 들어올 때 기존 값을 대체(overwrite)하는 것이 아니라 기존 리스트에 추가(append)되는 방식으로 동작
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## 노드(Node) 정의

1. **Agent**: 어떤 행동을 취할지(있다면) 결정하는 역할을 담당합니다.
2. **A function to invoke tool**: 에이전트가 사용하기 위해 호출한 함수. 각 노드에서 해당 액션 수행.

1. **Conditional Edge**: 특정 조건을 만족하는 경우에만 특정 노드로 이동하는 역할을 담당
2. **Normal Edge**: 도구 호출이 일어난 후, 항상 다음 노드로 이동하는 역할.

In [11]:
from langgraph.prebuilt import ToolNode

# 도구 실행 class
# 도구 호출 후 결과를 반환하는 역할
tool_executor = ToolNode(tools) 

# 에이전트 정의
def run_agent(data):
    agent_outcome = agent.invoke(data)
    return {"agent_outcome": agent_outcome}

# 도구 실행 함수 정의
def execute_tool(data):
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}

# conditional edge 결정 logic 정의
def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return 'end'
    else:
        return 'continue'

## Graph 정의

In [12]:
from langgraph.graph import END, StateGraph

# 새로운 그래프 정의
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tool)

# 조건부 엣지 추가
workflow.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})

# 일반 엣지 추가
workflow.add_edge("action", "agent")

# 그래프 시작점 정의
workflow.set_entry_point("agent")

# 그래프 컴파일
app = workflow.compile()

In [13]:
input = {"input": "What is the weather in sf", "chat_history": []}
for s in app.stream(input):
    print(list(s.values())[0])
    print('---')

{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'current weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'current weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 91, 'total_tokens': 113, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'function_call', 'logprobs': None}, id='run-5607ffd8-f9fa-4979-830a-c5ab0499d31b-0', usage_metadata={'input_tokens': 91, 'output_tokens': 22, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}, 'output_token_details